# Plasticity
This example shows how any material following the
[`MaterialModelsBase.jl`](https://github.com/KnutAM/MaterialModelsBase.jl)
interface can be assembled with `FerriteAssembly.jl`.
`element_routine!`)
has already been implemented for this material. This example shows how to
* Assemble materials with state variables
* Update state variables for the next time step

## Implementation
We start by the required packages

In [1]:
using Tensors, MaterialModelsBase, Ferrite, FerriteAssembly

And then we define the `material_response` for the plasticity material in
[`J2Plasticity.jl`](J2Plasticity.jl), which is basically the same as in
[`Ferrite.jl`'s plasticity example](https://ferrite-fem.github.io/Ferrite.jl/stable/examples/plasticity/)

In [2]:
include("J2Plasticity.jl");

## Assembly
With all required functions defined, we can now setup and assemble the finite element problem

In [3]:
material = J2Plasticity(200.0e9, 0.3, 200.0e6, 10.0e9);
grid = generate_grid(Tetrahedron, (20,2,4), zero(Vec{3}), Vec((10.0,1.0,1.0)));
cellvalues = CellVectorValues(
    QuadratureRule{3,RefTetrahedron}(2), Lagrange{3, RefTetrahedron, 1}());
dh = DofHandler(grid); add!(dh, :u, 3); close!(dh); # Create dofhandler
K = create_sparsity_pattern(dh);
r = zeros(ndofs(dh));

Using the `setup_assembly` function,

In [4]:
buffer, states_old, states_new = setup_assembly(dh, material, cellvalues)

(FerriteAssembly.DomainBuffer{false, FerriteAssembly.SubDofHandler{Ferrite.DofHandler{3, Ferrite.Grid{3, Ferrite.Tetrahedron, Float64}}, Nothing}, Vector{Int64}, FerriteAssembly.CellBuffer{3, Float64, Ferrite.CellVectorValues{3, Float64, Ferrite.RefTetrahedron, 9}, NamedTuple{(:u,), Tuple{UnitRange{Int64}}}, Main.var"##313".J2Plasticity{Float64, Tensors.SymmetricTensor{4, 3, Float64, 36}}, Vector{Main.var"##313".MaterialState{Float64, Tensors.SymmetricTensor{2, 3, Float64, 6}}}, Nothing, Nothing}, Nothing}(FerriteAssembly.SubDofHandler{Ferrite.DofHandler{3, Ferrite.Grid{3, Ferrite.Tetrahedron, Float64}}, Nothing}(Ferrite.DofHandler{3, Ferrite.Grid{3, Ferrite.Tetrahedron, Float64}}([:u], [3], Ferrite.Interpolation[Ferrite.Lagrange{3, Ferrite.RefTetrahedron, 1}()], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  690, 877, 878, 879, 943, 944, 945, 940, 941, 942], [1, 13, 25, 37, 49, 61, 73, 85, 97, 109  …  11413, 11425, 11437, 11449, 11461, 11473, 11485, 11497, 11509, 11521], Ferrite.ScalarWrapper{Boo

we setup the `buffer`, old state variables, and new state variables.
The state variables are created via the `create_cell_state`
function that is already defined for `MaterialModelsBase.AbstractMaterial`

We can now just provide an initial guess for the degree of freedom vector,`a`,
and do the assembly

In [5]:
a = zeros(ndofs(dh))
doassemble!(K, r, states_new, states_old, buffer; a=a);

If we would have a rate-dependent material, such that the time increment mattered,
we can also supply that (but that is not required in this example)

In [6]:
doassemble!(K, r, states_new, states_old, buffer; a=a, Δt=1.0);

In a full FE-program we iterate until convergence to find `a`. When converged,
we go to the next time step, and would like to set the old state equal to the
current state, which we can do by calling

In [7]:
update_states!(states_old, states_new)

If we would like to access the states in any cell, states_old and states_new can be
indexed with the cell number.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*